In [1]:
import configargparse
import torch
import torch.optim as optim
from environments import CartPoleRegulatorEnv
from environments import CartEnv
from environments import AcrobotEnv
from models.agents import NFQAgent
from models.networks import NFQNetwork, ContrastiveNFQNetwork
from util import get_logger, close_logger, load_models, make_reproducible, save_models
import matplotlib.pyplot as plt
import numpy as np
import itertools
import seaborn as sns

In [2]:
# Setup environment
bg_cart_mass = 1.0
fg_cart_mass = 1.0
init_experience = 400
force_left = 5
is_contrastive = False

train_env_bg = CartPoleRegulatorEnv(group=0, masscart=bg_cart_mass, mode="train", force_left=force_left, is_contrastive=is_contrastive)
train_env_fg = CartPoleRegulatorEnv(group=1, masscart=fg_cart_mass, mode="train", force_left=force_left, is_contrastive=is_contrastive)
eval_env_bg = CartPoleRegulatorEnv(group=0, masscart=bg_cart_mass, mode="eval", force_left=force_left, is_contrastive=is_contrastive)
eval_env_fg = CartPoleRegulatorEnv(group=1, masscart=fg_cart_mass, mode="eval", force_left=force_left, is_contrastive=is_contrastive)

# NFQ Main loop
bg_rollouts = []
fg_rollouts = []
total_cost = 0
if init_experience > 0:
    for _ in range(init_experience):
        rollout_bg, episode_cost = train_env_bg.generate_rollout(
            None, render=False, group=0
        )
        rollout_fg, episode_cost = train_env_fg.generate_rollout(
            None, render=False, group=1
        )
        bg_rollouts.extend(rollout_bg)
        fg_rollouts.extend(rollout_fg)
        total_cost += episode_cost

bg_rollouts.extend(fg_rollouts)
all_rollouts = bg_rollouts.copy()

bg_rollouts_test = []
fg_rollouts_test = []
if init_experience > 0:
    for _ in range(init_experience):
        rollout_bg, episode_cost = eval_env_bg.generate_rollout(
            None, render=False, group=0
        )
        rollout_fg, episode_cost = eval_env_fg.generate_rollout(
            None, render=False, group=1
        )
        bg_rollouts_test.extend(rollout_bg)
        fg_rollouts_test.extend(rollout_fg)
bg_rollouts_test.extend(fg_rollouts)
all_rollouts_test = bg_rollouts_test.copy()

/home/amandyam/.conda/envs/research/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [3]:
epoch = 3000
bg_success_queue = [0] * 3
fg_success_queue = [0] * 3
bg_converged = False

# Setup agent
nfq_net = ContrastiveNFQNetwork(state_dim=train_env_bg.state_dim, is_contrastive=is_contrastive)

optimizer = optim.Adam(nfq_net.parameters(), lr=1e-1)
nfq_agent = NFQAgent(nfq_net, optimizer)
nfq_agent._nfq_net.freeze_last_layers()
for ep in range(epoch + 1):
    
    if bg_converged:
        state_action_b, target_q_values, groups = nfq_agent.generate_pattern_set(fg_rollouts)
    else:
        state_action_b, target_q_values, groups = nfq_agent.generate_pattern_set(bg_rollouts)
    
    loss = nfq_agent.train((state_action_b, target_q_values, groups))
    
    eval_episode_length_fg = 0
    eval_episode_length_bg = 0
    eval_success_fg = False
    eval_success_bg = False
    if bg_converged:
        eval_episode_length_fg, eval_success_fg, eval_episode_cost_fg = nfq_agent.evaluate(
            eval_env_fg, render=False
        )
    else:
        eval_episode_length_bg, eval_success_bg, eval_episode_cost_bg = nfq_agent.evaluate(
        eval_env_bg, render=False
        )

    bg_success_queue = bg_success_queue[1:]
    bg_success_queue.append(1 if eval_success_bg else 0)

    fg_success_queue = fg_success_queue[1:]
    fg_success_queue.append(1 if eval_success_fg else 0)

    if sum(bg_success_queue) == 3:
        nfq_agent._nfq_net.unfreeze_last_layers()
        bg_converged = True

    if sum(fg_success_queue) == 3:
        print("FG Converged")
        break
    
    print("Epoch: " + str(ep) + " BG Converged: " + str(bg_converged) + " Eval BG: " + str(eval_episode_length_bg)
         + " Eval FG: " + str(eval_episode_length_fg))

Epoch: 0 BG Converged: False Eval BG: 22 Eval FG: 0
Epoch: 1 BG Converged: False Eval BG: 27 Eval FG: 0
Epoch: 2 BG Converged: False Eval BG: 38 Eval FG: 0
Epoch: 3 BG Converged: False Eval BG: 37 Eval FG: 0
Epoch: 4 BG Converged: False Eval BG: 31 Eval FG: 0
Epoch: 5 BG Converged: False Eval BG: 22 Eval FG: 0
Epoch: 6 BG Converged: False Eval BG: 27 Eval FG: 0
Epoch: 7 BG Converged: False Eval BG: 25 Eval FG: 0
Epoch: 8 BG Converged: False Eval BG: 27 Eval FG: 0
Epoch: 9 BG Converged: False Eval BG: 22 Eval FG: 0
Epoch: 10 BG Converged: False Eval BG: 20 Eval FG: 0
Epoch: 11 BG Converged: False Eval BG: 20 Eval FG: 0
Epoch: 12 BG Converged: False Eval BG: 20 Eval FG: 0
Epoch: 13 BG Converged: False Eval BG: 20 Eval FG: 0
Epoch: 14 BG Converged: False Eval BG: 22 Eval FG: 0
Epoch: 15 BG Converged: False Eval BG: 26 Eval FG: 0
Epoch: 16 BG Converged: False Eval BG: 21 Eval FG: 0
Epoch: 17 BG Converged: False Eval BG: 25 Eval FG: 0
Epoch: 18 BG Converged: False Eval BG: 24 Eval FG: 0
Epo

Epoch: 154 BG Converged: False Eval BG: 19 Eval FG: 0
Epoch: 155 BG Converged: False Eval BG: 21 Eval FG: 0
Epoch: 156 BG Converged: False Eval BG: 28 Eval FG: 0
Epoch: 157 BG Converged: False Eval BG: 25 Eval FG: 0
Epoch: 158 BG Converged: False Eval BG: 20 Eval FG: 0
Epoch: 159 BG Converged: False Eval BG: 24 Eval FG: 0
Epoch: 160 BG Converged: False Eval BG: 24 Eval FG: 0
Epoch: 161 BG Converged: False Eval BG: 22 Eval FG: 0
Epoch: 162 BG Converged: False Eval BG: 24 Eval FG: 0
Epoch: 163 BG Converged: False Eval BG: 22 Eval FG: 0
Epoch: 164 BG Converged: False Eval BG: 27 Eval FG: 0
Epoch: 165 BG Converged: False Eval BG: 25 Eval FG: 0
Epoch: 166 BG Converged: False Eval BG: 20 Eval FG: 0
Epoch: 167 BG Converged: False Eval BG: 24 Eval FG: 0
Epoch: 168 BG Converged: False Eval BG: 20 Eval FG: 0
Epoch: 169 BG Converged: False Eval BG: 20 Eval FG: 0
Epoch: 170 BG Converged: False Eval BG: 22 Eval FG: 0
Epoch: 171 BG Converged: False Eval BG: 20 Eval FG: 0
Epoch: 172 BG Converged: Fal

Epoch: 307 BG Converged: False Eval BG: 20 Eval FG: 0
Epoch: 308 BG Converged: False Eval BG: 22 Eval FG: 0
Epoch: 309 BG Converged: False Eval BG: 23 Eval FG: 0
Epoch: 310 BG Converged: False Eval BG: 26 Eval FG: 0
Epoch: 311 BG Converged: False Eval BG: 23 Eval FG: 0
Epoch: 312 BG Converged: False Eval BG: 21 Eval FG: 0
Epoch: 313 BG Converged: False Eval BG: 23 Eval FG: 0
Epoch: 314 BG Converged: False Eval BG: 25 Eval FG: 0
Epoch: 315 BG Converged: False Eval BG: 24 Eval FG: 0
Epoch: 316 BG Converged: False Eval BG: 23 Eval FG: 0
Epoch: 317 BG Converged: False Eval BG: 21 Eval FG: 0
Epoch: 318 BG Converged: False Eval BG: 22 Eval FG: 0
Epoch: 319 BG Converged: False Eval BG: 22 Eval FG: 0
Epoch: 320 BG Converged: False Eval BG: 22 Eval FG: 0
Epoch: 321 BG Converged: False Eval BG: 26 Eval FG: 0
Epoch: 322 BG Converged: False Eval BG: 22 Eval FG: 0
Epoch: 323 BG Converged: False Eval BG: 24 Eval FG: 0
Epoch: 324 BG Converged: False Eval BG: 21 Eval FG: 0
Epoch: 325 BG Converged: Fal

Epoch: 460 BG Converged: False Eval BG: 24 Eval FG: 0
Epoch: 461 BG Converged: False Eval BG: 20 Eval FG: 0
Epoch: 462 BG Converged: False Eval BG: 22 Eval FG: 0
Epoch: 463 BG Converged: False Eval BG: 22 Eval FG: 0
Epoch: 464 BG Converged: False Eval BG: 19 Eval FG: 0
Epoch: 465 BG Converged: False Eval BG: 21 Eval FG: 0
Epoch: 466 BG Converged: False Eval BG: 26 Eval FG: 0
Epoch: 467 BG Converged: False Eval BG: 23 Eval FG: 0
Epoch: 468 BG Converged: False Eval BG: 25 Eval FG: 0
Epoch: 469 BG Converged: False Eval BG: 25 Eval FG: 0
Epoch: 470 BG Converged: False Eval BG: 22 Eval FG: 0
Epoch: 471 BG Converged: False Eval BG: 23 Eval FG: 0
Epoch: 472 BG Converged: False Eval BG: 24 Eval FG: 0
Epoch: 473 BG Converged: False Eval BG: 20 Eval FG: 0
Epoch: 474 BG Converged: False Eval BG: 22 Eval FG: 0
Epoch: 475 BG Converged: False Eval BG: 25 Eval FG: 0
Epoch: 476 BG Converged: False Eval BG: 20 Eval FG: 0
Epoch: 477 BG Converged: False Eval BG: 21 Eval FG: 0
Epoch: 478 BG Converged: Fal

Epoch: 612 BG Converged: False Eval BG: 21 Eval FG: 0
Epoch: 613 BG Converged: False Eval BG: 23 Eval FG: 0
Epoch: 614 BG Converged: False Eval BG: 23 Eval FG: 0
Epoch: 615 BG Converged: False Eval BG: 20 Eval FG: 0
Epoch: 616 BG Converged: False Eval BG: 26 Eval FG: 0
Epoch: 617 BG Converged: False Eval BG: 22 Eval FG: 0
Epoch: 618 BG Converged: False Eval BG: 20 Eval FG: 0
Epoch: 619 BG Converged: False Eval BG: 20 Eval FG: 0
Epoch: 620 BG Converged: False Eval BG: 20 Eval FG: 0
Epoch: 621 BG Converged: False Eval BG: 20 Eval FG: 0
Epoch: 622 BG Converged: False Eval BG: 26 Eval FG: 0
Epoch: 623 BG Converged: False Eval BG: 26 Eval FG: 0
Epoch: 624 BG Converged: False Eval BG: 23 Eval FG: 0
Epoch: 625 BG Converged: False Eval BG: 25 Eval FG: 0
Epoch: 626 BG Converged: False Eval BG: 27 Eval FG: 0
Epoch: 627 BG Converged: False Eval BG: 22 Eval FG: 0
Epoch: 628 BG Converged: False Eval BG: 23 Eval FG: 0
Epoch: 629 BG Converged: False Eval BG: 20 Eval FG: 0
Epoch: 630 BG Converged: Fal

...

Epoch: 9298 BG Converged: False Eval BG: 24 Eval FG: 0
Epoch: 9299 BG Converged: False Eval BG: 22 Eval FG: 0
Epoch: 9300 BG Converged: False Eval BG: 28 Eval FG: 0
Epoch: 9301 BG Converged: False Eval BG: 24 Eval FG: 0
Epoch: 9302 BG Converged: False Eval BG: 27 Eval FG: 0
Epoch: 9303 BG Converged: False Eval BG: 23 Eval FG: 0
Epoch: 9304 BG Converged: False Eval BG: 24 Eval FG: 0
Epoch: 9305 BG Converged: False Eval BG: 24 Eval FG: 0
Epoch: 9306 BG Converged: False Eval BG: 26 Eval FG: 0
Epoch: 9307 BG Converged: False Eval BG: 24 Eval FG: 0
Epoch: 9308 BG Converged: False Eval BG: 26 Eval FG: 0
Epoch: 9309 BG Converged: False Eval BG: 27 Eval FG: 0
Epoch: 9310 BG Converged: False Eval BG: 25 Eval FG: 0
Epoch: 9311 BG Converged: False Eval BG: 28 Eval FG: 0
Epoch: 9312 BG Converged: False Eval BG: 23 Eval FG: 0
Epoch: 9313 BG Converged: False Eval BG: 24 Eval FG: 0
Epoch: 9314 BG Converged: False Eval BG: 28 Eval FG: 0
Epoch: 9315 BG Converged: False Eval BG: 21 Eval FG: 0
Epoch: 931

Epoch: 9448 BG Converged: False Eval BG: 28 Eval FG: 0
Epoch: 9449 BG Converged: False Eval BG: 20 Eval FG: 0
Epoch: 9450 BG Converged: False Eval BG: 20 Eval FG: 0
Epoch: 9451 BG Converged: False Eval BG: 20 Eval FG: 0
Epoch: 9452 BG Converged: False Eval BG: 24 Eval FG: 0
Epoch: 9453 BG Converged: False Eval BG: 27 Eval FG: 0
Epoch: 9454 BG Converged: False Eval BG: 28 Eval FG: 0
Epoch: 9455 BG Converged: False Eval BG: 28 Eval FG: 0
Epoch: 9456 BG Converged: False Eval BG: 26 Eval FG: 0
Epoch: 9457 BG Converged: False Eval BG: 22 Eval FG: 0
Epoch: 9458 BG Converged: False Eval BG: 22 Eval FG: 0
Epoch: 9459 BG Converged: False Eval BG: 26 Eval FG: 0
Epoch: 9460 BG Converged: False Eval BG: 19 Eval FG: 0
Epoch: 9461 BG Converged: False Eval BG: 24 Eval FG: 0
Epoch: 9462 BG Converged: False Eval BG: 23 Eval FG: 0
Epoch: 9463 BG Converged: False Eval BG: 24 Eval FG: 0
Epoch: 9464 BG Converged: False Eval BG: 24 Eval FG: 0
Epoch: 9465 BG Converged: False Eval BG: 24 Eval FG: 0
Epoch: 946

Epoch: 9597 BG Converged: False Eval BG: 28 Eval FG: 0
Epoch: 9598 BG Converged: False Eval BG: 20 Eval FG: 0
Epoch: 9599 BG Converged: False Eval BG: 20 Eval FG: 0
Epoch: 9600 BG Converged: False Eval BG: 21 Eval FG: 0
Epoch: 9601 BG Converged: False Eval BG: 23 Eval FG: 0
Epoch: 9602 BG Converged: False Eval BG: 24 Eval FG: 0
Epoch: 9603 BG Converged: False Eval BG: 20 Eval FG: 0
Epoch: 9604 BG Converged: False Eval BG: 26 Eval FG: 0
Epoch: 9605 BG Converged: False Eval BG: 27 Eval FG: 0
Epoch: 9606 BG Converged: False Eval BG: 21 Eval FG: 0
Epoch: 9607 BG Converged: False Eval BG: 23 Eval FG: 0
Epoch: 9608 BG Converged: False Eval BG: 20 Eval FG: 0
Epoch: 9609 BG Converged: False Eval BG: 24 Eval FG: 0
Epoch: 9610 BG Converged: False Eval BG: 21 Eval FG: 0
Epoch: 9611 BG Converged: False Eval BG: 27 Eval FG: 0
Epoch: 9612 BG Converged: False Eval BG: 21 Eval FG: 0
Epoch: 9613 BG Converged: False Eval BG: 22 Eval FG: 0
Epoch: 9614 BG Converged: False Eval BG: 23 Eval FG: 0
Epoch: 961

Epoch: 9747 BG Converged: False Eval BG: 22 Eval FG: 0
Epoch: 9748 BG Converged: False Eval BG: 22 Eval FG: 0
Epoch: 9749 BG Converged: False Eval BG: 24 Eval FG: 0
Epoch: 9750 BG Converged: False Eval BG: 20 Eval FG: 0
Epoch: 9751 BG Converged: False Eval BG: 21 Eval FG: 0
Epoch: 9752 BG Converged: False Eval BG: 24 Eval FG: 0
Epoch: 9753 BG Converged: False Eval BG: 19 Eval FG: 0
Epoch: 9754 BG Converged: False Eval BG: 23 Eval FG: 0
Epoch: 9755 BG Converged: False Eval BG: 23 Eval FG: 0
Epoch: 9756 BG Converged: False Eval BG: 28 Eval FG: 0
Epoch: 9757 BG Converged: False Eval BG: 25 Eval FG: 0
Epoch: 9758 BG Converged: False Eval BG: 27 Eval FG: 0
Epoch: 9759 BG Converged: False Eval BG: 23 Eval FG: 0
Epoch: 9760 BG Converged: False Eval BG: 21 Eval FG: 0
Epoch: 9761 BG Converged: False Eval BG: 26 Eval FG: 0
Epoch: 9762 BG Converged: False Eval BG: 23 Eval FG: 0
Epoch: 9763 BG Converged: False Eval BG: 21 Eval FG: 0
Epoch: 9764 BG Converged: False Eval BG: 21 Eval FG: 0
Epoch: 976

Epoch: 9897 BG Converged: False Eval BG: 20 Eval FG: 0
Epoch: 9898 BG Converged: False Eval BG: 22 Eval FG: 0
Epoch: 9899 BG Converged: False Eval BG: 25 Eval FG: 0
Epoch: 9900 BG Converged: False Eval BG: 25 Eval FG: 0
Epoch: 9901 BG Converged: False Eval BG: 25 Eval FG: 0
Epoch: 9902 BG Converged: False Eval BG: 22 Eval FG: 0
Epoch: 9903 BG Converged: False Eval BG: 22 Eval FG: 0
Epoch: 9904 BG Converged: False Eval BG: 21 Eval FG: 0
Epoch: 9905 BG Converged: False Eval BG: 26 Eval FG: 0
Epoch: 9906 BG Converged: False Eval BG: 19 Eval FG: 0
Epoch: 9907 BG Converged: False Eval BG: 19 Eval FG: 0
Epoch: 9908 BG Converged: False Eval BG: 21 Eval FG: 0
Epoch: 9909 BG Converged: False Eval BG: 24 Eval FG: 0
Epoch: 9910 BG Converged: False Eval BG: 24 Eval FG: 0
Epoch: 9911 BG Converged: False Eval BG: 22 Eval FG: 0
Epoch: 9912 BG Converged: False Eval BG: 23 Eval FG: 0
Epoch: 9913 BG Converged: False Eval BG: 21 Eval FG: 0
Epoch: 9914 BG Converged: False Eval BG: 26 Eval FG: 0
Epoch: 991